In [ ]:
from tools import PBE_solver
import bempp.api
bempp.api.PLOT_BACKEND = "gmsh"

### Forces for a sphere charge centered

In [8]:
sim = PBE_solver.Molecule('pqr_files\\sphere','sphere','point',3.0)
sim.get_solv_energy()
sim.get_solv_forces()
sim.get_fixed_forces()
sim.save_info(forces=True,energy=True)

C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


### Forces for arginine 

In [13]:
sim = PBE_solver.Molecule('pqr_files\\arg','arg','amber',3.0)
sim.get_solv_energy()
sim.get_solv_forces()
sim.get_fixed_forces()
sim.save_info(forces=True,energy=True)

C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


### Forces for two spheres charge centered with variable distance

In [4]:
distances = [[80,0,0],[40,0,0],[35,0,0],[30,0,0],[25,0,0],[20,0,0],[18,0,0],[16,0,0],[14,0,0],[12,0,0],[10,0,0],[9,0,0],[8,0,0],[7,0,0],[6,0,0],[5,0,0],[4,0,0]]
for distance in distances:
    sim = PBE_solver.Two_molecules('pqr_files\\sphere','pqr_files\\sphere','sphere','point',3.0,'sphere','point',3.0,distance,ep_in=2.0,ep_ex=80.0,kappa=0.125)
    sim.get_solv_energy()
    sim.get_solv_forces()
    sim.get_fixed_forces()
    sim.save_info(forces=True,energy=True)

C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


### Forces for arginine with methanol in variable distance

In [6]:
distances = [[80,0,0],[40,0,0],[35,0,0],[30,0,0],[25,0,0],[20,0,0],[18,0,0],[16,0,0],[14,0,0],[12,0,0],[10,0,0],[9,0,0],[8,0,0],[7,0,0],[6,0,0],[5,0,0],[4,0,0]]
for distance in distances:
    sim = PBE_solver.Two_molecules('pqr_files\\arg','pqr_files\\methanol','arg','amber',3.0,'methanol','amber',3.0,distance,ep_in=2.0,ep_ex=80.0,kappa=0.125)
    sim.get_solv_energy()
    sim.get_solv_forces()
    sim.get_fixed_forces()
    sim.save_info(forces=True,energy=True)

C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


### Forces for arginine and methanol (charges off) (Test Charge method)

In [9]:
distances = [[80,0,0],[40,0,0],[35,0,0],[30,0,0],[25,0,0],[20,0,0],[18,0,0],[16,0,0],[14,0,0],[12,0,0],[10,0,0],[9,0,0],[8,0,0],[7,0,0],[6,0,0],[5,0,0],[4,0,0]]
for distance in distances:
    sim = PBE_solver.Two_molecules('pqr_files\\arg','pqr_files\\methanol','arg','amber',3.0,'methanol','amber_nocharge',3.0,distance,ep_in=2.0,ep_ex=80.0,kappa=0.125)
    sim.get_solv_energy()
    sim.get_solv_forces()
    sim.get_fixed_forces()
    sim.save_info(forces=True,energy=True)

C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
